In [1]:
import pandas as pd

In [2]:
data = pd.read_excel("../datasets/dataset.xlsx",dtype={'CustomerID': str},parse_dates=['InvoiceDate'])

In [3]:
data.shape

(541909, 8)

# 1. Data Preprocessing 

## Making `Sale` column

In [4]:
data["Sale"] = (data['Quantity'] * data['UnitPrice']).round(2)

## Dropping Unwanted columns

In [5]:
data.drop(columns=["Description","UnitPrice","Quantity","CustomerID","InvoiceNo","Country","StockCode"], inplace=True)

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Ensure datetime index and set daily frequency
data_daily_sales.index = pd.to_datetime(data_daily_sales.index)
data_daily_sales = data_daily_sales.asfreq('D')

# Normalize sales data (Min-Max Scaling)
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data_daily_sales[['Sale']])

# Function to create input-output sequences
def create_sequences(data, time_steps=7):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i: i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

# Define time steps (how many past days to consider)
time_steps = 7  # Using past 7 days to predict the next day

# Create sequences
X, y = create_sequences(data_scaled, time_steps)

# Split data into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape input for LSTM (samples, time_steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build LSTM model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(time_steps, 1)),
    LSTM(50, activation='relu'),
    Dense(25, activation='relu'),
    Dense(1)  # Output layer
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

# Predict next 100 days
def forecast_future(model, data, num_days=100):
    future_preds = []
    last_sequence = data[-time_steps:].reshape(1, time_steps, 1)  # Get last available sequence

    for _ in range(num_days):
        pred = model.predict(last_sequence)[0][0]  # Predict next step
        future_preds.append(pred)

        # Update sequence with new prediction
        last_sequence = np.roll(last_sequence, -1)
        last_sequence[0, -1, 0] = pred  # Replace last value with predicted one

    # Convert predictions back to original scale
    future_preds = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1))
    
    # Create date index for predictions
    start_date = data_daily_sales.index.max() + pd.Timedelta(days=1)
    date_range = pd.date_range(start=start_date, periods=num_days, freq='D')

    return pd.DataFrame(future_preds, index=date_range, columns=['Predicted Sales'])

# Get predictions
future_sales = forecast_future(model, data_scaled, num_days=100)
print(future_sales.head())


C:\Users\hcham\anaconda3\Lib\site-packages\numpy\_typing\_scalars.py:12: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  _BoolLike_co = Union[bool, np.bool]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\hcham\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\hcham\AppData\Local\Temp\ipykernel_27780\2544866835.py", line 3, in <module>
    import tensorflow as tf
  File "C:\Users\hcham\anaconda3\Lib\site-packages\tensorflow\__init__.py", line 49, in <module>
    from tensorflow._api.v2 import __internal__
  File "C:\Users\hcham\anaconda3\Lib\site-packages\tensorflow\_api\v2\__internal__\__init__.py", line 8, in <module>
    from tensorflow._api.v2.__internal__ import autograph
  File "C:\Users\hcham\anaconda3\Lib\site-packages\tensorflow\_api\v2\__internal__\autograph\__init__.py", line 8, in <module>
    from tensorflow.python.autograph.core.ag_ctx import control_status_ctx # line: 34
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hcham\anaconda3\Lib\site-packages\te